In [ ]:
from helpers import *
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, RobertaTokenizer, LongformerForSequenceClassification
from torchmetrics import F1, Recall, Precision, PrecisionRecallCurve

In [ ]:
dataset_path = 'data/'
output_path = 'output/longformer-reuters-binary'
model_save_dir = os.path.join('saved-models/longformer-reuters-binary')
all_topics = ['earn']
num_labels = len(all_topics)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
filenames = get_filenames(dataset_path)

In [ ]:
dataset_train_dict, dataset_val_dict, dataset_test_dict = build_dataset_dictionaries(dataset_path, 
                                                                                     filenames, 
                                                                                     all_topics)

In [ ]:
csv_train, csv_eval, csv_test = write_to_csv(dataset_path, 
                                             all_topics, 
                                             dataset_train_dict, 
                                             dataset_val_dict, 
                                             dataset_test_dict)

In [ ]:
raw_datasets = load_dataset('csv', data_files={'train': os.path.join(dataset_path, csv_train),
                                               'eval': os.path.join(dataset_path, csv_eval),
                                               'test': os.path.join(dataset_path, csv_train)})
raw_datasets

In [ ]:
show_histogram_binary(all_topics[0], raw_datasets)

In [ ]:
tokenized_datasets = adjust_and_tokenize_datasets(raw_datasets)

In [ ]:
training_args = TrainingArguments(output_path, 
                                  evaluation_strategy="epoch", 
                                  save_strategy="epoch",
                                  per_device_train_batch_size=8, 
                                  per_device_eval_batch_size=8,
                                  num_train_epochs=5,
                                  learning_rate=4e-5,
                                  logging_steps=1000,
                                  load_best_model_at_end=True)


In [ ]:
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', 
                                                            ignore_mismatched_sizes=True)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['eval'], 
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained(model_save_dir)

In [ ]:
model = LongformerForSequenceClassification.from_pretrained(model_save_dir)
model.config
trainer = Trainer(
    model,
    tokenizer=tokenizer,
)

In [ ]:
predictions = trainer.predict(tokenized_datasets['test'])

In [ ]:
preds = torch.sigmoid(torch.tensor(predictions.predictions))
threshold = torch.tensor([0.5])
predicted_labels = (preds>threshold).float()*1
target = torch.tensor(predictions.label_ids, dtype=torch.int8)

In [ ]:
f1 = F1(num_classes=1, average=None, multiclass=False)
precision = Precision(num_classes=1, average=None, multiclass=False)
recall = Recall(num_classes=1, average=None, multiclass=False)
#curve = PrecisionRecallCurve()
print("f1 | precision | recall")
f1(predicted_labels, target).item(), precision(predicted_labels, target).item(), recall(predicted_labels, target).item()